<a href="https://colab.research.google.com/github/aslestia/ACS_2025/blob/main/ACS_week2_Gridworlds_DP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Pseudocode — GridWorld lengkap (sampai DP)
ENV:
  sama seperti (1), bisa tambah slip probability (transisi probabilistik).
  TRANSITIONS(s, a) → daftar (p, s', r)

VALUE ITERATION:
  Inisialisasi V(s) = 0
  repeat:
    for setiap state s (non-terminal):
      V_new(s) = max_a Σ_{s'} p(s'|s,a) [ r(s,a,s') + γ V(s') ]
    cek konvergensi ||V_new - V|| ≤ ε
    V ← V_new
  Ekstrak policy: π(s) = argmax_a Σ p(s'|s,a)[ r + γ V(s') ]

POLICY ITERATION:
  Inisialisasi π acak
  repeat:
    (Evaluasi)  hitung V^π dengan: V(s) = Σ p [ r + γ V(s') ] untuk aksi π(s)
    (Perbaikan) π_baru(s) = argmax_a Σ p [ r + γ V^π(s') ]
  until π_baru = π

In [ ]:
#Code TODO — GridWorld lengkap (tugas)
#Isi baris bertanda # TODO.

import numpy as np

ACTIONS = {0:(-1,0), 1:(0,1), 2:(1,0), 3:(0,-1)}   # U,R,D,L
ARROWS  = {0:"↑", 1:"→", 2:"↓", 3:"←", -1:"■"}

class GridWorld:
    def __init__(self, H=4, W=4, start=(0,0), goals={(3,3):+10}, traps={(1,2):-10,(2,1):-10},
                 step_reward=-1.0, gamma=0.95, slip=0.0):
        self.H,self.W = H,W; self.start=start
        self.goals, self.traps = goals, traps
        self.step_reward = step_reward; self.gamma = gamma; self.slip = slip
        self.nS, self.nA = H*W, 4

    def idx(self,s): return s[0]*self.W + s[1]
    def state_from_idx(self,i): return (i//self.W, i%self.W)
    def in_bounds(self,r,c): return 0<=r<self.H and 0<=c<self.W
    def is_terminal(self,s): return s in self.goals or s in self.traps

    def transitions(self, s, a):
        """Kembalikan list (p, s_next, reward)."""
        if self.is_terminal(s):
            return [(1.0, s, 0.0)]
        dr,dc = ACTIONS[a]
        cand = (s[0]+dr, s[1]+dc)
        if not self.in_bounds(*cand): cand = s

        left, right = (a-1)%4, (a+1)%4
        candL = (s[0]+ACTIONS[left][0], s[1]+ACTIONS[left][1])
        candR = (s[0]+ACTIONS[right][0], s[1]+ACTIONS[right][1])
        candL = candL if self.in_bounds(*candL) else s
        candR = candR if self.in_bounds(*candR) else s

        p_main = 1.0 - 2*self.slip
        outcomes = [(p_main,cand),(self.slip,candL),(self.slip,candR)]

        ts=[]
        for p,sn in outcomes:
            if p<=0: continue
            r = self.goals.get(sn, self.traps.get(sn, self.step_reward))
            ts.append((p,sn,r))
        return ts

def print_values(env, V):
    grid = np.zeros((env.H, env.W))
    for i in range(env.nS):
        r,c = env.state_from_idx(i); grid[r,c] = V[i]
    for r in range(env.H):
        print(" | ".join(f"{grid[r,c]:4.1f}" for c in range(env.W)))
    print()

def print_policy(env, Pi):
    for r in range(env.H):
        row=[]
        for c in range(env.W):
            s=(r,c)
            row.append(ARROWS[-1] if env.is_terminal(s) else ARROWS[Pi[env.idx(s)]])
        print(" ".join(row))
    print()

# ---------- TODO 1: Value Iteration ----------
def value_iteration(env, eps=1e-6, max_iter=1000):
    V = np.zeros(env.nS)
    iters=0
    for it in range(max_iter):
        iters=it+1; delta=0.0
        V_new = V.copy()
        for i in range(env.nS):
            s = env.state_from_idx(i)
            if env.is_terminal(s):
                V_new[i] = 0.0
                continue
            # TODO: hitung nilai Bellman optimal untuk state s
            qbest = -1e18
            for a in range(env.nA):
                q = 0.0
                for p,sn,r in env.transitions(s,a):
                    q += p * ( r + env.gamma * V[ env.idx(sn) ] )
                qbest = max(qbest, q)
            V_new[i] = qbest
            delta = max(delta, abs(V_new[i]-V[i]))
        V = V_new
        if delta < eps: break

    # TODO: ekstrak policy greedy dari V
    Pi = np.full(env.nS, -1, int)
    for i in range(env.nS):
        s = env.state_from_idx(i)
        if env.is_terminal(s): continue
        best_a, best_q = 0, -1e18
        for a in range(env.nA):
            q=0.0
            for p,sn,r in env.transitions(s,a):
                q += p * ( r + env.gamma * V[env.idx(sn)] )
            if q>best_q: best_q, best_a = q, a
        Pi[i] = best_a
    return V, Pi, iters

# ---------- TODO 2: Policy Evaluation ----------
def policy_evaluation(env, Pi, eps=1e-6, max_iter=1000):
    V = np.zeros(env.nS)
    for _ in range(max_iter):
        delta=0.0
        for i in range(env.nS):
            s = env.state_from_idx(i)
            if env.is_terminal(s): newv = 0.0
            else:
                a = Pi[i]; newv = 0.0
                # TODO: evaluasi nilai mengikuti aksi a
                for p,sn,r in env.transitions(s,a):
                    newv += p * ( r + env.gamma * V[ env.idx(sn) ] )
            delta = max(delta, abs(newv - V[i]))
            V[i] = newv
        if delta < eps: break
    return V

# ---------- TODO 3: Policy Improvement ----------
def policy_improvement(env, V):
    Pi = np.full(env.nS, -1, int)
    for i in range(env.nS):
        s = env.state_from_idx(i)
        if env.is_terminal(s): continue
        # TODO: pilih aksi terbaik berdasar V
        best_a, best_q = 0, -1e18
        for a in range(env.nA):
            q=0.0
            for p,sn,r in env.transitions(s,a):
                q += p * ( r + env.gamma * V[ env.idx(sn) ] )
            if q>best_q: best_q, best_a = q, a
        Pi[i] = best_a
    return Pi

def policy_iteration(env, eps=1e-6, max_iter=100):
    # inisialisasi policy (misal semua 'Up') kecuali terminal
    Pi = np.zeros(env.nS, dtype=int)
    for i in range(env.nS):
        if env.is_terminal(env.state_from_idx(i)): Pi[i] = -1
    iters=0
    for k in range(max_iter):
        iters=k+1
        V = policy_evaluation(env, Pi, eps=eps)
        newPi = policy_improvement(env, V)
        if np.array_equal(newPi, Pi): return V, Pi, iters
        Pi = newPi
    return V, Pi, iters

# ---------- Blok eksperimen (tinggal jalankan) ----------
env = GridWorld()
V_vi, Pi_vi, it_vi = value_iteration(env)
print("Value Iteration — Values:"); print_values(env, V_vi)
print("Value Iteration — Policy:"); print_policy(env, Pi_vi)
print("Iterasi:", it_vi)

V_pi, Pi_pi, it_pi = policy_iteration(env)
print("Policy Iteration — Values:"); print_values(env, V_pi)
print("Policy Iteration — Policy:"); print_policy(env, Pi_pi)
print("Iterasi:", it_pi)